<a href="https://colab.research.google.com/github/EmmanuelUCSP/Grafica-Final/blob/main/Implementacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


import os

video_dir = '/content/drive/MyDrive/AQA_videos'
files = os.listdir(video_dir)
print("Archivos encontrados:", files)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, ReLU, AveragePooling3D, Concatenate
from tensorflow.keras.layers import GlobalAveragePooling3D, Flatten, Dense
print("TensorFlow version:", tf.__version__)


# ***Conv3D-AQA***

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, ReLU, AveragePooling3D, Concatenate
from tensorflow.keras.layers import GlobalAveragePooling3D, Dense

def build_conv3d_aqa(input_shape=(16, 112, 112, 3)):
    inputs = Input(shape=input_shape)

    # Rama 1
    x1 = Conv3D(32, kernel_size=(3, 3, 3), padding='same')(inputs)
    x1 = ReLU()(x1)

    # Rama 2
    x2 = Conv3D(32, kernel_size=(6, 3, 3), padding='same')(inputs)
    x2 = ReLU()(x2)

    # Rama 3 + Average Pooling (sin reducir dimensiones)
    x3 = Conv3D(32, kernel_size=(3, 3, 3), padding='same')(inputs)
    x3 = ReLU()(x3)
    x3 = AveragePooling3D(pool_size=(1, 1, 1), padding='same')(x3)

    # Concatenación de ramas
    x = Concatenate()([x1, x2, x3])

    # Bloques intermedios
    x = Conv3D(64, kernel_size=(4, 3, 3), padding='same')(x)
    x = ReLU()(x)

    x = Conv3D(64, kernel_size=(6, 3, 3), padding='same')(x)
    x = ReLU()(x)

    x = Conv3D(64, kernel_size=(1, 1, 1), padding='same')(x)
    x = ReLU()(x)

    # Fully connected
    x = GlobalAveragePooling3D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)

    # Salida: score único
    outputs = Dense(1, activation='linear')(x)

    return Model(inputs, outputs)



# ***Preprocesamiento***

In [ ]:
import random

video_dir = '/content/drive/MyDrive/AQA_videos'
video_files = []
for root, _, files in os.walk(video_dir):
    for f in files:
        if f.endswith('.avi'):
            full_path = os.path.join(root, f)
            video_files.append(full_path)


score_dict = {}
for video_path in video_files:
    # video_path: /content/drive/MyDrive/AQA_videos/diving/001.avi
    relative_path = os.path.relpath(video_path, video_dir)  # diving/001.avi
    score = round(random.uniform(2.0, 5.0), 2)
    score_dict[relative_path] = score

print(score_dict)

In [ ]:
import cv2
import numpy as np

def load_video_tensor(video_path, num_frames=16, size=(112, 112)):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames < num_frames:
        print(f"⚠️ Video con pocos frames: {video_path} ({total_frames} frames)")
        return None

    selected_idxs = np.linspace(0, total_frames - 1, num_frames).astype(int)
    frames = []
    frame_idx = 0
    selected_pos = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx == selected_idxs[selected_pos]:
            frame = cv2.resize(frame, size)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = frame / 255.0  # Normalización
            frames.append(frame)
            selected_pos += 1

            if selected_pos == len(selected_idxs):
                break

        frame_idx += 1

    cap.release()

    if len(frames) != num_frames:
        print(f"❌ No se lograron extraer 16 frames de: {video_path}")
        return None

    return np.array(frames)


# ***Compilar modelo***

In [ ]:
model = build_conv3d_aqa()
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()


# ***Train***

In [20]:
X = []
y = []

for relative_path, score in score_dict.items():
    full_path = os.path.join(video_dir, relative_path)
    tensor = load_video_tensor(full_path)

    if tensor is not None and tensor.shape == (16, 112, 112, 3):
        X.append(tensor)
        y.append(score)
    else:
        print(f"❌ Video descartado: {relative_path}")

X = np.array(X)
y = np.array(y).reshape(-1, 1)

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (1189, 16, 112, 112, 3)
y shape: (1189, 1)


In [ ]:
history = model1.fit(X, y, epochs=20, batch_size=2, validation_split=0.2)


In [ ]:
model1.save('/content/conv3d_aqa_model.h5')


# ***Pruebas***

In [ ]:
from tensorflow.keras.losses import MeanSquaredError

model1 = load_model('/content/conv3d_aqa_model.h5', custom_objects={'mse': MeanSquaredError()})

In [ ]:
# Ejemplo
test_video_path = "/content/drive/MyDrive/AQA_videos/gym_vault/001.avi"

test_tensor = load_video_tensor(test_video_path)

if test_tensor is None:
    print("❌ Error cargando el video.")
else:
    # Asegura la forma (1, 16, 112, 112, 3)
    test_tensor = np.expand_dims(test_tensor, axis=0)

    # Predecir
    prediction = model.predict(test_tensor)
    print(f"🎯 Predicción de score: {prediction[0][0]:.2f}")


In [ ]:
import numpy as np
print("Máximo y mínimo de y:", np.max(y), np.min(y))


In [ ]:
history = model.fit(...)  # ← si lo guardaste

import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title("Evolución de pérdida")
plt.show()
